LSM EXP 05_Aiffel
<br/>**5. 인공지능과 가위바2위보 하기**

데이터 정보
<br/>데이터 가져오기
<br/>모델 형성
<br/>*하이퍼 파라미터 조정
<br/>*하이퍼 파라미터 조정(drop rate 제외)

#가위바위보 데이터와 Sequential API

In [56]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

**tensorflow**는 구글이 개발한 오픈소스 소프트웨어 딥러닝 및 머신러닝 라이브러리이다. 수학 계산식과 데이터의 흐름을 노드와 엣지를 사용한 방향성 그래프, 데이터 플로우 그래프로 나타낸다.


**Keras**는 Tensorflow 위에서 동작하는 라이브러리이다.
<br/>사용자 친화적으로 개발된 Keras의 쉽다는 장점과
<br/>딥러닝 프로젝트에서 범용적으로 활용할 수 있는
<br/>Tensorflow의 장점을 통합할 수 있는 환경을 설정한다.

**numpy**는 array 단위로 벡터와 행렬을 계산한다.


**sklearn.model_selection**은 훈련 데이터와 테스트 데이터를 분리한다.

**matplotlib**은 다양한 데이터와 학습 모델을 시각화한다.

**os(Operating System)**는 운영체제에서 제공되는 여러 기능을 파이썬에서 수행한다. <br/>예를 들어, 파일 복사, 디렉터리 생성, 파일 목록을 구할 수 있다.

In [3]:
from PIL import Image
import glob

**PIL(Python Image Library)**는 다양한 이미지 파일 형식을 지원하는 작업 모듈이다. 다만, PIL의 지원이 2011년 중단되고, Pillow가 PIL의 후속 프로젝트로 나왔다.


**glob**는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다. 단, 조건에 정규식을 사용할 수 없으며 엑셀 등에서도 사용할 수 있는 '*'와 '?'같은 와일드카드만을 지원한다.

##데이터 정보

**rock_sissor_paper**

2021년 아이펠(Aiffel) 수강생이 촬영했던 가위, 바위, 보 이미지를 통합하여 만든 데이터셋이다.
<br/>포토윅스라는 프로그램을 이용하여 이미지를 편집했다.
<br/>분석하기에 좋다.

이 데이터셋은 224X224 pixel의 337개의 이미지로 구성되어 있다.
<br/>가위(114개), 바위(112개), 보(111개)의 이미지는 가위바위보 게임의 동작을 나타낸다.


##데이터 가져오기

In [4]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.

**왜 28X28인가?**
<br/>'Why 28x28 pixel'라고 검색해보았으나 나오지 않는다.
<br/>784(28 x 28)개의 각 뉴런들은 각 픽셀의 밝기를 나타낸다고만 검색 결과가 나올 뿐이다.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [59]:
image_dir_path1 = '/content/drive/MyDrive/LMS/rock_scissor_paper/scissor/'
resize_images(image_dir_path1)

2320  images to be resized.
2320  images resized.


가위 이미지를 28x28 사이즈로 바꾼다.

In [65]:
image_dir_path2 = '/content/drive/MyDrive/LMS/rock_scissor_paper/rock/'
resize_images(image_dir_path2)

2202  images to be resized.
2202  images resized.


바위 이미지를 28x28 사이즈로 바꾼다.

In [66]:
image_dir_path3 = '/content/drive/MyDrive/LMS/rock_scissor_paper/paper/'
resize_images(image_dir_path3)

2270  images to be resized.
2270  images resized.


보 이미지를 28x28 사이즈로 바꾼다.

for문을 실행하기 앞서 np.zeros와 reshape를 통해 행렬을 초기화한다.
<br/>**np.zeros**는 크기(number_of_data*img_size*img_size*color)만큼의 0으로만 채워진 1차원 벡터를 생성한다.

**reshape**은 크기(number_of_data,img_size,img_size,color)만큼의 3차원 텐서를 생성한다.

In [71]:
def load_rock_paper_scissors_data(data_dir, width=224, height=224):
    x_data = []
    y_data = []
    dirs = os.listdir(data_dir)
    dirs = [dir for dir in dirs if not dir.startswith ('.')] #.DS_Store 제외
    for dir in dirs:
        #print(dir) #paper
        files = os.listdir(os.path.join(data_dir, dir))
        files = [file for file in files if not file.startswith ('.')] #.DS_Store 제외
        for file in files:
            #print(file) #8.jpg
            x_data.append(one_image(os.path.join(data_dir, dir, file), width, height))
            if dir == 'rock':
                y_data.append(0)
            elif dir == 'paper':
                y_data.append(1)
            elif dir == 'scissors':
                y_data.append(2)
    x_data = np.array(x_data)
    y_data = np.array(y_data)

    return x_data, y_data

In [74]:
def one_image(image_file, width=224, height=224):
    image = Image.open(image_file)
    image = image.resize((224, 224))
    image = np.array(image)
    image = image/ 255.0 

    return image

라벨을 가위(0), 바위(1), 보(2)로 한다.
<br/>이미지와 라벨을 담은 행렬을 생성한다.
<br/>255로 나누어 입력은 0~1 사이의 값으로 정규화한다.


In [75]:
x_data, y_data = load_rock_paper_scissors_data('/content/drive/MyDrive/LMS/rock_scissor_paper')
labels = ['rock', 'scissor', 'paper']

x_data, y_data를 생성한다.

<br/>훈련 데이터와 테스트 데이터를 7:3으로 맞춘다.
<br/>훈련 데이터의 크기를 6792의 70%인 4755장으로 맞춘다.

In [62]:
y_data = tf.keras.utils.to_categorical(y_data)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777, stratify=y_data)

ValueError: ignored

In [50]:
print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

IndexError: ignored

##모델 형성

###하이퍼 파라미터 조정

In [36]:
n_channel_1 = 256
n_channel_2 = 512
n_dense = 512
n_class = 3
n_drop_rate = 0.3
n_train_epoch = 10

레이어의 개수, 분류 클래스, drop rate, 최적화의 학습단위(train epoch) 등 하이퍼파라미터 튜닝을 한다.

**Drop-out**은 서로 연결된 연결망(layer)에서 0부터 1 사이의 확률로 뉴런을 제거(drop)하는 기법이다.
<br/>Drop-out을 적용하여 상관관계가 강한 Feature를 제외하여
<br/>해당 Feature에만 출력값이 좌지우지되는 과대적합(overfitting)을 방지한다.

In [37]:
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Dropout(n_drop_rate))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Dropout(n_drop_rate))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(n_class, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))

Model에 추가된 Layer 개수:  9


첫번째 레이어는 사이즈 3의 256개의 필터로 구성되어 있다. 이미지 형태는 28X28 크기이다.
<br/>relu 함수는 활성화함수로 구성된다.
<br/>2 x 2 max-pooling 레이어를 가진다. 추상화된 형태를 오버피팅을 방지하는데 도움을 준다.

In [38]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 26, 26, 256)       7168      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 13, 13, 256)      0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 13, 13, 256)       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 11, 11, 512)       1180160   
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 5, 5, 512)        0         
 g2D)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 5, 5, 512)        

In [39]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(x_train_norm, y_train, epochs= n_train_epoch)

Epoch 1/10
149/149 [==============================] - 2s 10ms/step - loss: 1.0046 - accuracy: 1.0000
Epoch 2/10
149/149 [==============================] - 1s 10ms/step - loss: 0.8341 - accuracy: 1.0000
Epoch 3/10
149/149 [==============================] - 1s 10ms/step - loss: 0.6917 - accuracy: 1.0000
Epoch 4/10
149/149 [==============================] - 1s 10ms/step - loss: 0.5746 - accuracy: 1.0000
Epoch 5/10
149/149 [==============================] - 1s 10ms/step - loss: 0.4793 - accuracy: 1.0000
Epoch 6/10
149/149 [==============================] - 1s 10ms/step - loss: 0.4021 - accuracy: 1.0000
Epoch 7/10
149/149 [==============================] - 1s 10ms/step - loss: 0.3396 - accuracy: 1.0000
Epoch 8/10
149/149 [==============================] - 1s 10ms/step - loss: 0.2888 - accuracy: 1.0000
Epoch 9/10
149/149 [==============================] - 1s 10ms/step - loss: 0.2474 - accuracy: 1.0000
Epoch 10/10
149/149 [==============================] - 1s 10ms/step - loss: 0.2134 - accura

loss이 지나치네 작고 accuracy가 지나치게 큰 이상한 학습 결과가 나온다.
<br/>따라서 drop_rate 하이퍼 파라미터를 제외하기로 했다.

###하이퍼 파라미터 조정(drop rate 제외)

In [40]:
n_channel_1 = 256
n_channel_2 = 512
n_channel_3 = 512
n_dense = 512
n_class = 3
n_train_epoch = 10

In [41]:
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPooling2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(n_class, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))

Model에 추가된 Layer 개수:  9


In [42]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 26, 26, 256)       7168      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 13, 13, 256)      0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 11, 11, 512)       1180160   
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 5, 5, 512)        0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 3, 3, 512)         2359808   
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 1, 1, 512)       

In [43]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(x_train_norm, y_train, epochs= n_train_epoch)

Epoch 1/10
149/149 [==============================] - 3s 12ms/step - loss: 1.0046 - accuracy: 1.0000
Epoch 2/10
149/149 [==============================] - 2s 12ms/step - loss: 0.8341 - accuracy: 1.0000
Epoch 3/10
149/149 [==============================] - 2s 12ms/step - loss: 0.6917 - accuracy: 1.0000
Epoch 4/10
149/149 [==============================] - 2s 11ms/step - loss: 0.5746 - accuracy: 1.0000
Epoch 5/10
149/149 [==============================] - 2s 11ms/step - loss: 0.4793 - accuracy: 1.0000
Epoch 6/10
149/149 [==============================] - 2s 11ms/step - loss: 0.4021 - accuracy: 1.0000
Epoch 7/10
149/149 [==============================] - 2s 12ms/step - loss: 0.3396 - accuracy: 1.0000
Epoch 8/10
149/149 [==============================] - 2s 12ms/step - loss: 0.2888 - accuracy: 1.0000
Epoch 9/10
149/149 [==============================] - 2s 11ms/step - loss: 0.2474 - accuracy: 1.0000
Epoch 10/10
149/149 [==============================] - 2s 11ms/step - loss: 0.2134 - accura

drop rate의 문제가 아니었다.
<br/>결과는 accuracy가 지나치게 크게 나온다.

하이퍼 파라미터 최적화 알고리즘으로 Adam을 사용한다.

[Adam]
<br/>모멘텀과 AdaGrad를 결합한다.
<br/>매개변수 공간을 효율적으로 탐색해주며
<br/>하이퍼파라미터의 '편향 보정'이 진행된다는 점이 Adam의 특징이다.

In [44]:
image_dir_path = '/content/drive/MyDrive/LMS/Rock'
(x_test, y_test)=load_data(image_dir_path, 2037)
x_test_norm = x_test/255.0   

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

학습데이터(x_train)의 이미지 개수는 0 입니다.
x_test shape: (2037, 28, 28, 3)
y_test shape: (2037,)


255로 나누어 입력은 0~1 사이의 값으로 정규화한다.
<br/>훈련 데이터와 테스트 데이터를 7:3으로 맞춘다.
<br/>훈련 데이터의 크기를 6792의 30%인 2037장으로 맞춘다.

In [45]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

64/64 - 1s - loss: 0.1983 - accuracy: 1.0000 - 627ms/epoch - 10ms/step
test_loss: 0.19830520451068878 
test_accuracy: 1.0
